In [38]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from math import sqrt
from scipy.special import erfc
from tqdm import tqdm_notebook

def getPrecise(t):
    res = np.zeros(N)
    res[0] = T0
    if t > 0:
        for i in range(N):
            res[i] = T0*erfc(i*dx/(2*sqrt(L*t*dt)))
        
    return res

def getPrecise_nelin(t):
    res = np.zeros(N)
#     res[0] = getT(dt*t)
    for i in range(N):
        if i*dx <= c*t*dt:
            res[i] = pow(sigma*c/L0*(c*t*dt-i*dx), 1/sigma)
            
    return res

# Линейное уравнение теплопроводности

$$
\frac{\partial{T}}{\partial{t}} = \lambda \frac{\partial^2{T}}{\partial{x^2}},  \lambda > 0
$$

In [39]:
L = 1 # lambda
T0 = 10

dt = 0.1
dx = 0.45

N = 70
T = 400

sigma = L*dt/dx**2
print("Число Куранта = ", sigma)

A = np.zeros((T,N))

for n in range(-1, T-1):
    A[n+1][0] = T0
    for i in range(1, N-1):
        A[n+1][i] = A[n][i] + sigma*(A[n][i+1]-2*A[n][i]+A[n][i-1])
    A[n+1][N-1] = 0    
        

fig, ax = plt.subplots(figsize = (10, 10))
xdata, ydata = range(N), []
ax.grid(True)
ln2, = plt.plot([], [], color = 'k', linestyle = ':', label = 'Точное решение')
ln, = plt.plot([], [], color = 'r', linestyle = ':', label = 'Явная схема')
plt.legend()

def init():
    ax.set_xlim(0, N-1)
    ax.set_ylim(0, T0*1.1)
    return ln,

def update(t):
    ydata = A[frame_const*t]
    ln.set_data(xdata, ydata)
    ln2.set_data(xdata, getPrecise(frame_const*t))
    return ln,

frame_const = 5
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=50, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

Число Куранта =  0.49382716049382713


In [40]:
thetta = 0.5

B = np.zeros((T,N))
alpha = np.zeros(N)
beta = np.zeros(N)

K = sigma
C = 2*sigma+1+thetta

sigma = L*dt/dx**2

for n in range(-1, T-1):
    B[n+1][0] = T0
    alpha[1] = 0
    beta[1] = T0
    for i in range(1, N-1):
        D = (1+2*thetta)*B[n][i]-thetta*B[n-1][i]
        alpha[i+1] = K/(C-K*alpha[i])
        beta[i+1] = (K*beta[i]+D)/(C-K*alpha[i])
    B[n+1][N-1] = 0
    for i in range(N-2, 0, -1):
        B[n+1][i] = alpha[i+1]*B[n+1][i+1] + beta[i+1]


fig, ax = plt.subplots(figsize = (10, 10))
ax.grid(True)
xdata, ydata = range(N), []
ln2, = plt.plot([], [], color = 'k', linestyle = ':', label = 'Точное решение')
ln, = plt.plot([], [], color = 'b', linestyle = '-.', label = 'Трёхслойная схема')
plt.legend()

def init():
    ax.set_xlim(0, N-1)
    ax.set_ylim(0, T0*1.1)
    return ln,

def update(t):
    ydata = B[frame_const*t]
    ln.set_data(xdata, ydata)
    ln2.set_data(xdata, getPrecise(frame_const*t))
    return ln,

frame_const = 5
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=50, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

### Сравнение

In [41]:
fig, ax = plt.subplots(figsize = (10, 10))
ax.grid(True)
xdata, ydata = range(N), []
ln, = plt.plot([], [], color = 'r', linestyle = '-', label = 'Явная схема')
ln2, = plt.plot([], [], color = 'b', linestyle = ':', label = 'Трёхслойная схема')
plt.legend()

def init():
    ax.set_xlim(0, N-1)
    ax.set_ylim(0, T0*1.1)
    return ln,

def update(t):
    ln.set_data(xdata, A[frame_const*t])
    ln2.set_data(xdata, B[frame_const*t])
    return ln,

frame_const = 5
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=50, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

# Нелинейное уравнение теплопроводности

$$
\frac{\partial{T}}{\partial{t}} = \frac{\partial}{\partial x} \lambda(T) \frac{\partial{T}}{\partial{x}},  \lambda > 0
$$

In [42]:
def getT(t):
    return T0 * pow(t, 1/sigma)

def getL(T):
    return L0 * pow(T, sigma)


T0 = 0.5
L0 = 0.5
sigma = 4

dt = 0.1
dx = 0.45

N = 35
T = 1800

c = sqrt(getL(T0)/sigma)
print("с = ", c)

courant = dt/dx**2
print("Число Куранта = ", courant)

с =  0.08838834764831845
Число Куранта =  0.49382716049382713


In [43]:
epsilon = 3

A = np.zeros((T,N))

for n in range(-1, T-1):
    A[n+1][0] = getT(dt*(n+1))
    for i in range(1, N-1):
        Li = getL(A[n][i])
        Lip = getL(A[n][i+1])
        Lin = getL(A[n][i-1])
        if dx*i-c*dt*n > -epsilon:
            Lp = (Li+Lip)/2
            Ln = (Li+Lin)/2
        else:
            Lp = 2*Li*Lip/(Li+Lip)
            Ln = 2*Li*Lin/(Li+Lin)            
            
        A[n+1][i] = A[n][i] + courant*(Lp*(A[n][i+1]-A[n][i]) - Ln*(A[n][i]-A[n][i-1]))
    A[n+1][N-1] = 0
        

fig, ax = plt.subplots(figsize = (10, 10))
xdata, ydata = range(N), []
ax.grid(True)
ln, = plt.plot([], [], color = 'g', linestyle = '-', label = 'Явная схема')
ln2, = plt.plot([], [], color = 'k', linestyle = ':', label = 'Точное решение')
plt.legend()

def init():
    ax.set_xlim(0, N-1)
    ax.set_ylim(0, T0*5)
    return ln,

def update(t):
    ydata = A[frame_const*t]
    ln.set_data(xdata, ydata)
    ln2.set_data(xdata, getPrecise_nelin(frame_const*t))
    ct = frame_const*t*dt*c/dx
    return ln,

frame_const = 10
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=30, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

In [44]:
epsilon = 1
thetta = 0.6

B = np.zeros((T,N))
alpha = np.zeros(N)
beta = np.zeros(N)

for n in range(-1, T-1):
    B[n+1][0] = getT(dt*(n+1))
    alpha[1] = 0
    beta[1] = B[n+1][0]
    for i in range(1, N-1):
        Li = getL(B[n][i])
        Lip = getL(B[n][i+1])
        Lin = getL(B[n][i-1])
        if dx*i-c*dt*n > -epsilon:
            Lp = (Li+Lip)/2
            Ln = (Li+Lin)/2
        else:
            Lp = 2*Li*Lip/(Li+Lip)
            Ln = 2*Li*Lin/(Li+Lin) 
        
        tA = courant * Ln
        tB = courant * Lp
        tC = tA+tB+(1+thetta)
        tD = (1+2*thetta)*B[n][i]-thetta*B[n-1][i]
        alpha[i+1] = tB/(tC-tA*alpha[i])
        beta[i+1] = (tA*beta[i]+tD)/(tC-tA*alpha[i])
    B[n+1][N-1] = 0
    for i in range(N-2, 0, -1):
        B[n+1][i] = alpha[i+1]*B[n+1][i+1] + beta[i+1]
        

fig, ax = plt.subplots(figsize = (10, 10))
xdata, ydata = range(N), []
ax.grid(True)
ln, = plt.plot([], [], color = 'r', linestyle = '-', label = 'Трёхслойная схема')
ln2, = plt.plot([], [], color = 'k', linestyle = ':', label = 'Точное решение')
plt.legend()

def init():
    ax.set_xlim(0, N-1)
    ax.set_ylim(0, T0*5)
    return ln,

def update(t):
    ydata = B[frame_const*t]
    ln.set_data(xdata, ydata)
    ln2.set_data(xdata, getPrecise_nelin(frame_const*t))
    ct = frame_const*t*dt*c/dx
    return ln,

frame_const = 15
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=30, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

In [45]:
fig, ax = plt.subplots(figsize = (10, 10))
ax.grid(True)
xdata, ydata = range(N), []
ln2, = plt.plot([], [], color = 'g', linestyle = '-', label = 'Явная схема')
ln, = plt.plot([], [], color = 'r', linestyle = '-', label = 'Трёхслойная схема')
ln3, = plt.plot([], [], color = 'k', linestyle = ':', label = 'Точное решение')
plt.legend()

def init():
    ax.set_xlim(0, N-1)
    ax.set_ylim(0, T0*5)
    return ln,

def update(t):
    ln.set_data(xdata, B[frame_const*t])
    ln2.set_data(xdata, A[frame_const*t])
    ln3.set_data(xdata, getPrecise_nelin(frame_const*t))
    return ln,

frame_const = 10
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=30, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)